In [1]:
from agential.cog.agent.reflexion import ReflexionCoTAgent, ReflexionReActAgent
from agential.cog.prompts.agent.reflexion import (
    REFLEXION_COT_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_COT_REFLECT,
    REFLEXION_COT_REFLECT_INSTRUCTION_HOTPOTQA,
    REFLEXION_REACT_INSTRUCTION_HOTPOTQA,
    REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA,
    HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_REACT_REFLECT
)
from agential.cog.prompts.benchmark.hotpotqa import (
    HOTPOTQA_FEWSHOT_EXAMPLES_COT
)

from langchain_openai import ChatOpenAI

import dotenv
dotenv.load_dotenv()

import warnings
warnings.filterwarnings('ignore')

llm = ChatOpenAI()

In [ ]:
question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"


agent = ReflexionCoTAgent(
    llm=llm,
    mode={"qa": "hotpotqa"},
)
out = agent.generate(
    question=question, 
    key=key, 
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    prompt=REFLEXION_COT_INSTRUCTION_HOTPOTQA, 
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_COT_REFLECT,
    reflect_prompt=REFLEXION_COT_REFLECT_INSTRUCTION_HOTPOTQA,
    reflection_strategy="reflexion",
    additional_keys={},
    reflection_additional_keys={},
    max_trials=3,
    patience=3,
    reset=True
)

In [ ]:
out

In [2]:
question = "VIVA Media AG changed it's name in 2004. What does their new acronym stand for?"
key = "Gesellschaft mit beschränkter Haftung"


agent = ReflexionReActAgent(
    llm=llm,
    mode={"qa": "hotpotqa"},
)
out = agent.generate(
    question=question, 
    key=key, 
    examples=HOTPOTQA_FEWSHOT_EXAMPLES_COT,
    prompt=REFLEXION_COT_INSTRUCTION_HOTPOTQA, 
    reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_REFLEXION_COT_REFLECT,
    reflect_prompt=REFLEXION_COT_REFLECT_INSTRUCTION_HOTPOTQA,
    reflection_strategy="reflexion",
    additional_keys={},
    reflection_additional_keys={},
    max_steps=6,
    max_trials=3,
    patience=3,
    reset=True
)

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\langchain_core\_api\deprecation.py:119: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


TypeError: ReflexionReActReflector.reflect() got an unexpected keyword argument 'additional_keys'